In [34]:
# Cell 1: 导入库
import cv2
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import re


In [15]:
!wget -O vis/videos/fisheye_night.mp4 "https://www.pexels.com/download/video/9575733/?fps=29.97&h=960&w=540"

--2025-09-19 17:40:02--  https://www.pexels.com/download/video/9575733/?fps=29.97&h=960&w=540
Resolving www.pexels.com (www.pexels.com)... 104.18.67.220, 104.18.66.220, 2606:4700::6812:43dc, ...
Connecting to www.pexels.com (www.pexels.com)|104.18.67.220|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://videos.pexels.com/video-files/9575733/9575733-sd_540_960_30fps.mp4 [following]
--2025-09-19 17:40:03--  https://videos.pexels.com/video-files/9575733/9575733-sd_540_960_30fps.mp4
Resolving videos.pexels.com (videos.pexels.com)... 104.18.67.220, 104.18.66.220, 2606:4700::6812:42dc, ...
Connecting to videos.pexels.com (videos.pexels.com)|104.18.67.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6833386 (6.5M) [video/mp4]
Saving to: ‘vis/videos/fisheye_night.mp4’

vis/videos/fisheye_ 100%[===================>]   6.52M  5.23MB/s    in 1.2s    

2025-09-19 17:40:05 (5.23 MB/s) - ‘vis/videos/fisheye_night.mp4’ saved [6833386/6

In [4]:
# Cell 2: 定义视频和输出参数
video_path = 'vis/videos/statue1.mp4'
temp_frame_dir = 'vis/temp_frames/'
os.makedirs(temp_frame_dir, exist_ok=True)



In [17]:
## ==============================================================================
#           ⭐ V6.3: 专业归档（含完整帧序列）⭐
# ==============================================================================

# 1. 导入和参数设置
import os
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import re
import subprocess # 用于更稳定地执行外部脚本
import shutil # 用于文件操作
import json # 用于生成参数信息文件

# ==============================================================================
# ▼▼▼ 核心参数设置 (您的设置已保留) ▼▼▼
video_path = 'vis/videos/fisheye_night.mp4' # 初始视频
style_dir = 'data_alt/style/'
final_save_dir = 'vis/final/'
# ==============================================================================

# --- 自动生成与检查路径 ---
video_name = os.path.splitext(os.path.basename(video_path))[0]
base_temp_dir = 'vis/temp_frames/'
base_final_dir = 'data_alt/content/'
temp_frame_dir = os.path.join(base_temp_dir, video_name)
final_input_dir = os.path.join(base_final_dir, video_name)
temp_gif_dir = 'vis/temp_gifs' # 存放临时预览GIF

# 创建所有需要的目录
for path in [style_dir, final_save_dir, temp_frame_dir, final_input_dir, temp_gif_dir]:
    os.makedirs(path, exist_ok=True)

print(f"输入视频: {video_path}")
print(f"内容帧保存目录: {final_input_dir}")
print(f"最终成品保存目录: {final_save_dir}")
print("-" * 20)

# --- 准备工作：解帧 ---
print("正在将视频解帧...")
subprocess.run(f'ffmpeg -i "{video_path}" -q:v 2 "{temp_frame_dir}/frame_%04d.png" -y', shell=True, check=True, capture_output=True)
frame_files = sorted([os.path.join(temp_frame_dir, f) for f in os.listdir(temp_frame_dir) if f.endswith('.png')])
total_frames = len(frame_files)
print(f"解帧完成，共 {total_frames} 帧。")

# --- 动态扫描风格文件 ---
style_files = [f for f in os.listdir(style_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
if not style_files:
    print(f"警告：在 {style_dir} 中未找到任何风格图片！")

# ==============================================================================
# 🔥🔥🔥 构建交互式UI控制台 🔥🔥🔥
# ==============================================================================

if total_frames > 0:
    # --- 状态变量 ---
    crop_preview_mode = False
    last_output_gif_path = ""
    last_input_gif_path = ""
    last_stylized_frame_path = "" # << 新增：用于存储结果帧的路径

    # --- UI 组件定义 ---
    frame_slider = widgets.IntSlider(min=1, max=total_frames, step=1, value=1, description='预览帧:', continuous_update=False, layout=widgets.Layout(width='80%'))
    frame_label = widgets.Label(value=f"当前: {frame_slider.value}/{total_frames}")
    preview_output = widgets.Output()
    start_frame_input = widgets.IntText(value=0, description='起始帧:', layout=widgets.Layout(width='150px'))
    second_frame_input = widgets.IntText(value=0, description='第二帧:', layout=widgets.Layout(width='150px'))
    total_frames_input = widgets.IntText(value=8, description='总帧数:', layout=widgets.Layout(width='150px'))
    autofill_button = widgets.Button(description="➡️ 自动填充帧列表")
    add_button = widgets.Button(description="✅ 手动添加此帧")
    clear_button = widgets.Button(description="❌ 清空列表")
    selected_frames_textarea = widgets.Textarea(value='', placeholder='这里显示您选择的帧号...', description='已选帧:', layout=widgets.Layout(width='90%', height='100px'))
    crop_x = widgets.IntText(value=224, description='裁剪X坐标:')
    crop_y = widgets.IntText(value=14, description='裁剪Y坐标:')
    crop_size = widgets.IntText(value=512, description='裁剪尺寸:')
    preview_crop_button = widgets.Button(description="✂️ 预览裁剪效果")
    preview_input_gif_button = widgets.Button(description="🎬 生成输入GIF预览")
    input_gif_output = widgets.Output()
    style_dropdown = widgets.Dropdown(options=style_files, description='选择风格:')
    generate_command_button = widgets.Button(description="📋 生成终端命令 📋", button_style='success')
    command_textarea = widgets.Textarea(value='', placeholder='点击上方按钮后，这里会生成可复制的命令...', description='终端命令:', layout=widgets.Layout(width='90%', height='100px'))
    generate_gif_button = widgets.Button(description="🖼️ 生成结果GIF", button_style='primary')
    output_gif_output = widgets.Output()
    save_button = widgets.Button(description="💾 专业归档保存 💾", button_style='info')
    save_output = widgets.Output()

    # --- 核心功能函数 ---
    def update_preview(change=None):
        global crop_preview_mode; frame_index = frame_slider.value; x, y, size = crop_x.value, crop_y.value, crop_size.value
        frame_label.value = f"当前: {frame_index}/{total_frames}"
        with preview_output:
            clear_output(wait=True); img = cv2.imread(frame_files[frame_index - 1]); img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            fig, ax = plt.subplots(figsize=(8, 5)); ax.imshow(img)
            if crop_preview_mode: ax.set_xlim(x, x + size); ax.set_ylim(y + size, y)
            else: rect = patches.Rectangle((x, y), size, size, linewidth=2, edgecolor='r', facecolor='none'); ax.add_patch(rect)
            ax.set_title(f"帧 {frame_index}"); ax.axis('off'); plt.show()
    def autofill_button_clicked(b):
        start, second, total = start_frame_input.value, second_frame_input.value, total_frames_input.value
        if start >= second: print("错误：第二帧必须大于起始帧！"); return
        interval = second - start; frames = [start]; current_frame = start
        for _ in range(total - 1): current_frame += interval; frames.append(current_frame)
        selected_frames_textarea.value = '\n'.join(map(str, frames))
    def process_selected_frames(output_dir):
        # 清空旧帧，确保每次都是新的开始
        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)
        os.makedirs(output_dir, exist_ok=True)
        frame_numbers_str = re.findall(r'\d+', selected_frames_textarea.value)
        if not frame_numbers_str: return []
        frame_numbers = [int(n) for n in frame_numbers_str]; x, y, size = crop_x.value, crop_y.value, crop_size.value
        for i, frame_num in enumerate(frame_numbers):
            if 1 <= frame_num <= total_frames:
                img = cv2.imread(frame_files[frame_num - 1]); cropped_img = img[y:y+size, x:x+size]
                cv2.imwrite(os.path.join(output_dir, f"{i+1:04d}.png"), cropped_img)
        return True
    def generate_and_show_gif(frame_folder, output_widget, gif_name="temp.gif"):
        output_widget.clear_output(); gif_path = os.path.join(temp_gif_dir, gif_name)
        with output_widget:
            print("正在生成GIF..."); subprocess.run(f'ffmpeg -framerate 10 -i "{frame_folder}/%04d.png" -y "{gif_path}"', shell=True, capture_output=True)
            print("GIF生成完毕！"); display(Image(filename=gif_path, width=400))
        return gif_path
    def preview_input_gif_clicked(b):
        global last_input_gif_path
        preview_frame_dir = os.path.join(temp_gif_dir, "input_preview_frames")
        process_selected_frames(preview_frame_dir)
        last_input_gif_path = generate_and_show_gif(preview_frame_dir, input_gif_output, f"{video_name}_input_preview.gif")
    def generate_command_button_clicked(b):
        command_textarea.value = ""; print(f"步骤1: 正在处理并保存内容帧到 {final_input_dir}...")
        if not process_selected_frames(final_input_dir): print("错误：已选帧列表为空，无法生成命令！"); return
        print("内容帧保存完毕。")
        style_name = style_dropdown.value; style_name_no_ext = os.path.splitext(style_name)[0]
        output_dir = f"./results/{video_name}/{style_name_no_ext}_stylized_{video_name}"
        python_command = ' '.join(['python', 'run_styleid_v2v.py', '--data_root', './data_alt', '--content_name', video_name, '--style_name', f"'{style_name}'", '--output_dir', f"'{output_dir}'"])
        command_textarea.value = python_command; print("✅ 终端命令已生成！请从下方的文本框中复制。")
    
    # << 修改：此函数现在会记录结果帧的路径 >>
    def generate_gif_button_clicked(b):
        global last_output_gif_path, last_stylized_frame_path
        style_name = style_dropdown.value; style_name_no_ext = os.path.splitext(style_name)[0]
        output_dir = f"./results/{video_name}/{style_name_no_ext}_stylized_{video_name}"
        print(f"正在检查目录: {output_dir}")
        if not os.path.exists(output_dir): print(f"❌ 错误：输出目录 '{output_dir}' 未找到。请确认您已在终端成功运行命令。"); return
        stylized_frame_folders = [d for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))]
        if not stylized_frame_folders: print(f"❌ 错误：在 '{output_dir}' 中找不到存放风格化后帧的子文件夹！"); return
        
        stylized_frame_path = os.path.join(output_dir, stylized_frame_folders[0])
        last_stylized_frame_path = stylized_frame_path # << 记录路径
        
        print(f"在 '{stylized_frame_path}' 中找到帧，开始生成GIF...")
        last_output_gif_path = generate_and_show_gif(stylized_frame_path, output_gif_output, f"{video_name}_stylized_by_{style_name_no_ext}.gif")

    # << 核心修改: 再次升级保存函数，增加保存帧序列的功能 >>
    def _perform_save(destination_folder):
        """实际执行文件复制和归档的函数"""
        style_image_name = style_dropdown.value
        style_image_path = os.path.join(style_dir, style_image_name)

        # 1. 创建所有归档目录结构
        result_dest_folder = os.path.join(destination_folder, 'result')
        content_dest_folder = os.path.join(destination_folder, 'content')
        style_dest_folder = os.path.join(destination_folder, 'style')
        frame_content_dest = os.path.join(destination_folder, 'frames', 'content')
        frame_result_dest = os.path.join(destination_folder, 'frames', 'result')
        
        for path in [result_dest_folder, content_dest_folder, style_dest_folder, frame_content_dest, frame_result_dest]:
            os.makedirs(path, exist_ok=True)

        print(f"创建归档文件夹: {destination_folder}")
        
        # 2. 创建并保存参数信息文件 (info.json)
        try:
            frame_numbers = [int(n) for n in re.findall(r'\d+', selected_frames_textarea.value)]
            metadata = { "source_info": { "original_video": video_path, "processed_frames": frame_numbers }, "style_info": { "style_image": style_dropdown.value }, "processing_parameters": { "crop": { "x": crop_x.value, "y": crop_y.value, "size": crop_size.value } }, "command_generated": command_textarea.value.strip() }
            info_file_path = os.path.join(destination_folder, 'info.json')
            with open(info_file_path, 'w', encoding='utf-8') as f: json.dump(metadata, f, ensure_ascii=False, indent=4)
            print(f"  - 参数信息已保存至 'info.json'")
        except Exception as e: print(f"  - ⚠️ 警告: 保存参数信息文件失败: {e}")

        # 3. 复制GIF和风格图
        shutil.copy(last_output_gif_path, result_dest_folder)
        print(f"  - 结果GIF已保存至 'result' 文件夹")
        if last_input_gif_path and os.path.exists(last_input_gif_path):
            shutil.copy(last_input_gif_path, content_dest_folder)
            print(f"  - 内容GIF已保存至 'content' 文件夹")
        shutil.copy(style_image_path, style_dest_folder)
        print(f"  - 风格图已保存至 'style' 文件夹")
        
        # 4. 【新增】复制内容帧和结果帧的完整序列
        try:
            shutil.copytree(final_input_dir, frame_content_dest, dirs_exist_ok=True)
            print(f"  - 内容帧已保存至 'frames/content'")
        except Exception as e: print(f"  - ⚠️ 警告: 复制内容帧失败: {e}")
            
        if last_stylized_frame_path and os.path.exists(last_stylized_frame_path):
            try:
                shutil.copytree(last_stylized_frame_path, frame_result_dest, dirs_exist_ok=True)
                print(f"  - 结果帧已保存至 'frames/result'")
            except Exception as e: print(f"  - ⚠️ 警告: 复制结果帧失败: {e}")
        else:
             print(f"  - ⚠️ 警告: 未找到结果帧目录，跳过保存。")

        print(f"\n✅ 专业归档完成！")

    def save_button_clicked(b):
        save_output.clear_output()
        with save_output:
            if not (last_output_gif_path and os.path.exists(last_output_gif_path)):
                print("❌ 错误: 未找到最终结果GIF。请先生成结果GIF。")
                return
            if not (last_input_gif_path and os.path.exists(last_input_gif_path)):
                print("⚠️ 警告: 未找到内容GIF。您可以先点击'生成输入GIF预览'来生成它。将只保存结果和风格图。")
            style_image_name = style_dropdown.value
            style_image_path = os.path.join(style_dir, style_image_name)
            if not os.path.exists(style_image_path):
                print(f"❌ 错误: 找不到风格图片 '{style_image_path}'。")
                return

            gif_basename = os.path.basename(last_output_gif_path)
            folder_name = os.path.splitext(gif_basename)[0]
            main_dest_folder = os.path.join(final_save_dir, folder_name)

            if not os.path.exists(main_dest_folder):
                _perform_save(main_dest_folder)
            else:
                print(f"检测到文件夹 '{folder_name}' 已存在。")
                i = 1
                while True:
                    suggested_name = f"{folder_name}_{i}"
                    if not os.path.exists(os.path.join(final_save_dir, suggested_name)):
                        break
                    i += 1

                rename_prompt = widgets.HTML(f"<b>请为本次归档提供一个新的文件夹名称：</b>")
                new_name_input = widgets.Text(value=suggested_name, description='新文件夹名:', layout=widgets.Layout(width='60%'))
                confirm_button = widgets.Button(description="✅ 确认保存", button_style='success')
                cancel_button = widgets.Button(description="❌ 取消")
                rename_output = widgets.Output()

                def on_confirm_button_clicked(cb):
                    rename_output.clear_output()
                    new_folder_name = new_name_input.value.strip()
                    if not new_folder_name:
                        with rename_output: print("❌ 错误: 文件夹名称不能为空。")
                        return
                    final_dest_folder = os.path.join(final_save_dir, new_folder_name)
                    if os.path.exists(final_dest_folder):
                        with rename_output: print(f"❌ 错误: 名称 '{new_folder_name}' 仍然存在，请选择一个不同的名称。")
                        return
                    save_output.clear_output()
                    with save_output: _perform_save(final_dest_folder)

                def on_cancel_button_clicked(cb):
                    save_output.clear_output()
                    with save_output: print("归档操作已取消。")

                confirm_button.on_click(on_confirm_button_clicked)
                cancel_button.on_click(on_cancel_button_clicked)
                rename_ui = widgets.VBox([rename_prompt, new_name_input, widgets.HBox([confirm_button, cancel_button]), rename_output])
                display(rename_ui)

    def toggle_crop_preview_clicked(b):
        global crop_preview_mode; crop_preview_mode = not crop_preview_mode
        b.description = "🖼️ 返回完整视图" if crop_preview_mode else "✂️ 预览裁剪效果"; update_preview()
    def add_frame_clicked(b):
        current_frame = str(frame_slider.value); current_list = list(filter(None, selected_frames_textarea.value.split('\n')))
        if current_frame not in current_list:
            current_list.append(current_frame); current_list.sort(key=int); selected_frames_textarea.value = '\n'.join(current_list)
    def clear_list_clicked(b): selected_frames_textarea.value = ''

    # --- 绑定所有功能到UI组件 ---
    frame_slider.observe(update_preview, names='value')
    crop_x.observe(update_preview, names='value'); crop_y.observe(update_preview, names='value'); crop_size.observe(update_preview, names='value')
    autofill_button.on_click(autofill_button_clicked); add_button.on_click(add_frame_clicked); clear_button.on_click(clear_list_clicked)
    preview_input_gif_button.on_click(preview_input_gif_clicked); preview_crop_button.on_click(toggle_crop_preview_clicked)
    generate_command_button.on_click(generate_command_button_clicked)
    generate_gif_button.on_click(generate_gif_button_clicked)
    save_button.on_click(save_button_clicked)
    
    # --- 最终UI布局 ---
    frame_selection_accordion = widgets.Accordion(children=[widgets.VBox([ widgets.HBox([start_frame_input, second_frame_input, total_frames_input]), autofill_button, widgets.HBox([add_button, clear_button]),])], titles=('高级选帧 (自动填充/手动)',))
    ui = widgets.VBox([
        widgets.HTML("<h2>步骤1: 准备内容素材</h2>"), widgets.HBox([frame_slider, frame_label]), preview_output, frame_selection_accordion, selected_frames_textarea,
        widgets.HTML("<hr>"), widgets.HBox([crop_x, crop_y, crop_size]), widgets.HBox([preview_crop_button, preview_input_gif_button]), input_gif_output,
        widgets.HTML("<h2>步骤2: 生成命令并手动执行</h2>"), style_dropdown, generate_command_button, command_textarea,
        widgets.HTML("<h2>步骤3: 生成并预览结果</h2>"), generate_gif_button, output_gif_output,
        widgets.HTML("<h2>步骤4: 专业归档保存</h2>"), save_button, save_output
    ])
    
    display(ui)
    update_preview()

else:
    print("错误：未能从视频中提取任何帧。")

输入视频: vis/videos/fisheye_night.mp4
内容帧保存目录: data_alt/content/fisheye_night
最终成品保存目录: vis/final/
--------------------
正在将视频解帧...
解帧完成，共 906 帧。
